<a href="https://colab.research.google.com/github/chanseongparkk/2022_Data_Analysis_Intern_practice/blob/main/07_27_01_%EB%82%98%EC%9D%B4%EB%B8%8C%EB%B2%A0%EC%9D%B4%EC%A6%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 나이브베이즈 실습

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [ ]:
## 가우지안 나이브 베이즈

from sklearn.naive_bayes import GaussianNB

## iris 데이터 로드

iris = load_iris()
df_x = pd.DataFrame(iris.data, columns = iris.feature_names)
df_y = pd.DataFrame(iris.target, columns = ["target"])

# train test split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 123)

print(X_train.shape, X_test.shape)


(120, 4) (30, 4)


In [ ]:
X_train.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
17,5.1,3.5,1.4,0.3
98,5.1,2.5,3.0,1.1
66,5.6,3.0,4.5,1.5
126,6.2,2.8,4.8,1.8
109,7.2,3.6,6.1,2.5


In [ ]:
##  가우지안 나이브베이즈 생성

gnb = GaussianNB()

## 모델링

fit = gnb.fit(X_train, y_train)
y_pred = fit.predict(X_test)
print(y_pred)

# 실제값과 예측값을 비교하여 틀린 갯수 확인
print("테스트 데이터 수: %d, 틀린 갯수 : %d"
%(X_test.shape[0],(np.array(y_test.target.tolist()) != y_pred).sum()))

[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 1 0 2 0 0 0 2 2 0]
테스트 데이터 수: 30, 틀린 갯수 : 1


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
## 예측한 클래스와 해당 예측 데이터의 클래스별 확률
# 클래스
print(fit.classes_)

# 분류된 클래스 수
print(fit.class_count_)

# 테스트 데이터
print(X_test[:1],"\n")

# 테스트 데이터 1개에 대한 클래스가 나온 확률
print(fit.predict_proba(X_test)[:1],"\n")

# 테스트 데이터 1개의 클래스
print(fit.predict(X_test)[:1],"\n")

[0 1 2]
[37. 44. 39.]
    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
72                6.3               2.5                4.9               1.5 

[[7.24143720e-126 9.23061979e-001 7.69380215e-002]] 

[1] 



In [ ]:
## 특정 값의 클래스별 확률 확인
print(fit.predict_proba(X_test)[[1,20]])

# 특정 변수의 범주별 클래스 확인
fit.predict(X_test)[[1,20]]

[[1.81805757e-197 1.22131524e-005 9.99987787e-001]
 [1.00000000e+000 8.57846082e-017 1.77097412e-025]]


array([2, 0])

In [ ]:
# confusion matrix 확인
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

# 2범주에 있는 하나의 오답이 오답의 전부이다.
# 나이브 베이즈는 투자대비 효용이 높은 간단한 머신러닝 모델

array([[13,  0,  0],
       [ 0,  6,  0],
       [ 0,  1, 10]])

### 나이브 베이즈 직접 계산

In [ ]:
X_test.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
72,6.3,2.5,4.9,1.5
112,6.8,3.0,5.5,2.1
132,6.4,2.8,5.6,2.2
88,5.6,3.0,4.1,1.3
37,4.9,3.6,1.4,0.1


In [ ]:
# 예측할 데이터
predict_data = np.array(X_test.iloc[0])
predict_data

array([6.3, 2.5, 4.9, 1.5])

In [ ]:
# 클래스 별 각 피쳐의 평균
fit.theta_

array([[5.01621622, 3.43243243, 1.46756757, 0.25945946],
       [5.95      , 2.78409091, 4.24090909, 1.32272727],
       [6.58717949, 2.95897436, 5.57948718, 2.02820513]])

In [ ]:
# 클래스 별 각 피쳐의 분산
fit.sigma_

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `sigma_` was deprecated in 1.0 and will be removed in1.2. Use `var_` instead.
  warnings.warn(msg, category=FutureWarning)


array([[0.10568298, 0.14975895, 0.02705625, 0.01214025],
       [0.27068182, 0.10042872, 0.22741736, 0.04221075],
       [0.39752795, 0.11011177, 0.29188692, 0.0774096 ]])

In [ ]:
# 클래스 별 사전 확률
fit.class_prior_

array([0.30833333, 0.36666667, 0.325     ])

In [ ]:
## class별 가능도 (likelihood) 산출
likelihood = [
# 0클래스
(sp.stats.norm(fit.theta_[0][0],np.sqrt(fit.sigma_[0][0])).pdf(predict_data[0])*\
 sp.stats.norm(fit.theta_[0][1],np.sqrt(fit.sigma_[0][1])).pdf(predict_data[1])*\
 sp.stats.norm(fit.theta_[0][2],np.sqrt(fit.sigma_[0][2])).pdf(predict_data[2])*\
 sp.stats.norm(fit.theta_[0][3],np.sqrt(fit.sigma_[0][3])).pdf(predict_data[3])),\
 
# 1클래스
(sp.stats.norm(fit.theta_[1][0],np.sqrt(fit.sigma_[1][0])).pdf(predict_data[0])*\
 sp.stats.norm(fit.theta_[1][1],np.sqrt(fit.sigma_[1][1])).pdf(predict_data[1])*\
 sp.stats.norm(fit.theta_[1][2],np.sqrt(fit.sigma_[1][2])).pdf(predict_data[2])*\
 sp.stats.norm(fit.theta_[1][3],np.sqrt(fit.sigma_[1][3])).pdf(predict_data[3])),\
 
# 2클래스
(sp.stats.norm(fit.theta_[2][0],np.sqrt(fit.sigma_[2][0])).pdf(predict_data[0])*\
 sp.stats.norm(fit.theta_[2][1],np.sqrt(fit.sigma_[2][1])).pdf(predict_data[1])*\
 sp.stats.norm(fit.theta_[2][2],np.sqrt(fit.sigma_[2][2])).pdf(predict_data[2])*\
 sp.stats.norm(fit.theta_[2][3],np.sqrt(fit.sigma_[2][3])).pdf(predict_data[3]))
]
 
likelihood

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `sigma_` was deprecated in 1.0 and will be removed in1.2. Use `var_` instead.
  warnings.warn(msg, category=FutureWarning)


[2.0700298536453225e-126, 0.2218869448618605, 0.020865556464324073]

In [ ]:
## 사후확률
posterior = likelihood * fit.class_prior_
print(posterior)

## 정규화
posterior / np.sum(posterior, axis = 0)

[6.38259205e-127 8.13585464e-002 6.78130585e-003]


array([7.24143720e-126, 9.23061979e-001, 7.69380215e-002])

In [ ]:
## scipy gnb를 사용한 추정값
print(fit.predict_proba(X_test)[[0]])

[[7.24143720e-126 9.23061979e-001 7.69380215e-002]]


### 다항분포 나이브 베이즈

* 독립변수가 범주형

In [ ]:
## multinomial naive bayes

from sklearn.naive_bayes import MultinomialNB

## 난수 생성(0-3, 변수 10개 샘플사이즈 4)

mnb_X_train = np.random.randint(4, size = (4,10))
mnb_y_train = np.array([1,2,3,4])


In [ ]:
print(mnb_X_train)
print(mnb_y_train)

[[0 1 1 3 0 0 0 1 0 1]
 [1 1 2 2 3 0 2 0 1 2]
 [1 1 1 0 3 1 1 2 2 0]
 [3 2 3 2 0 0 0 0 2 0]]
[1 2 3 4]


In [ ]:
# 학습
mnb = MultinomialNB()

fit_mnb = mnb.fit(mnb_X_train, mnb_y_train)

print(fit_mnb.predict(mnb_X_train)[:1]) #예측값
print(fit_mnb.predict_proba(mnb_X_train)[:1]) #확률

[1]
[[0.912042   0.03872709 0.00175825 0.04747266]]


#### Prior(사전 확률 ) 설정
- class 가 발생되는 사전 확률을 미리 알고 있었던 경우라고 가정

In [ ]:
mnb2 = MultinomialNB(class_prior = [0.2,0.5,0.1,0.2])
mnb2.fit(mnb_X_train,mnb_y_train)

MultinomialNB(class_prior=[0.2, 0.5, 0.1, 0.2])

In [ ]:
mnb2.predict_proba(mnb_X_train)

array([[8.62686401e-01, 9.15783956e-02, 8.31549161e-04, 4.49036547e-02],
       [7.52498985e-03, 9.87969369e-01, 2.44373738e-03, 2.06190342e-03],
       [3.84395178e-03, 8.83064811e-02, 9.03145683e-01, 4.70388377e-03],
       [3.73348393e-03, 9.04591980e-03, 3.80725259e-04, 9.86839871e-01]])